# Simulación de la Venta de Boletos en el Cine Center

In [3]:
!pip install simpy

In [4]:
import random
import numpy
import simpy

## Datos de la simulación

In [5]:

SEMILLA = 40                  # Semilla generador
CLIENTES = 100                # vamos a simular 10 clientes
LLEGADA_CLIENTES = [0, 120]   # Clientes llegan cada 10 segundos en una distribución uniforme. 
ATENCION_CLIENTES = [40, 100] # Clientes son atendidos.

# Variables de desempeño
COLA = 0
MAX_COLA = 0
ESPERA_CLIENTES = numpy.array([])


## Definición de Funciones

In [6]:
def llegada(env, numero, contador):
    for i in range(numero):
        c = cliente(env, 'Cliente %02d' % i, contador)
        env.process(c)
        tiempo_llegada = random.uniform(LLEGADA_CLIENTES[0], LLEGADA_CLIENTES[1])
        yield env.timeout(tiempo_llegada) # Yield retorna un objeto iterable

In [7]:
def cliente(env, nombre, servidor):
    # El cliente llega y se va cuando es atendido
    llegada = env.now
    print('%7.2f'%(env.now), " Llega el cliente ", nombre)
    global COLA
    global MAX_COLA
    global ESPERA_CLIENTES
    # Atendemos a los clientes (retorno del yield)
    # With ejecuta un iterador sin importar si hay excepciones o no
    with servidor.request() as req:
        # Hacemos la espera hasta que sea atendido el cliente
        COLA += 1
        if COLA > MAX_COLA:
            MAX_COLA = COLA
            
        # print("Tamaño de cola", COLA)
        results = yield req
        COLA = COLA - 1
        espera = env.now - llegada
        ESPERA_CLIENTES = numpy.append(ESPERA_CLIENTES, espera)
        
        print('%7.2f'%(env.now), " El cliente ", nombre, " espera a ser atendido ", espera)
        
        tiempo_atencion = random.uniform(ATENCION_CLIENTES[0], ATENCION_CLIENTES[1])
        yield env.timeout(tiempo_atencion)
        
        print('%7.2f'%(env.now), " Sale el cliente ", nombre)
        

## Inicio de la Simulación

In [9]:
print("Sala de Cine")
random.seed(SEMILLA)
env = simpy.Environment()

# Inicio del proceso y ejecución
servidor = simpy.Resource(env, capacity=2)
env.process(llegada(env, CLIENTES, servidor))
env.run()

print("Cola Máxima ", MAX_COLA)
print("Tiempo promedio de espera ", '%7.2f'%(numpy.mean(ESPERA_CLIENTES)))


Sala de Cine
   0.00  Llega el cliente  Cliente 00
   0.00  El cliente  Cliente 00  espera a ser atendido  0
  55.03  Llega el cliente  Cliente 01
  55.03  El cliente  Cliente 01  espera a ser atendido  0.0
  58.85  Llega el cliente  Cliente 02
  92.67  Sale el cliente  Cliente 00
  92.67  El cliente  Cliente 02  espera a ser atendido  33.81769144881534
 111.98  Sale el cliente  Cliente 01
 172.53  Sale el cliente  Cliente 02
 174.27  Llega el cliente  Cliente 03
 174.27  El cliente  Cliente 03  espera a ser atendido  0.0
 189.67  Llega el cliente  Cliente 04
 189.67  El cliente  Cliente 04  espera a ser atendido  0.0
 235.17  Sale el cliente  Cliente 03
 256.12  Sale el cliente  Cliente 04
 295.24  Llega el cliente  Cliente 05
 295.24  El cliente  Cliente 05  espera a ser atendido  0.0
 298.68  Llega el cliente  Cliente 06
 298.68  El cliente  Cliente 06  espera a ser atendido  0.0
 314.26  Llega el cliente  Cliente 07
 377.16  Sale el cliente  Cliente 06
 377.16  El cliente  Cliente 